In [13]:
import cv2
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.neighbors import NearestNeighbors
import os
from glob import glob

def load_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img, gray

def keypoint_graph(image_path, label):
    img, gray = load_image(image_path)
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(gray, None)

    if des is None or len(kp) == 0:
        return Data(x=torch.zeros((0, 128)), edge_index=torch.empty((2, 0), dtype=torch.long)), img, []

    points = np.array([k.pt for k in kp])
    x = torch.tensor(des, dtype=torch.float)

    k = min(5, len(points))
    nbrs = NearestNeighbors(n_neighbors=k).fit(points)
    adjacency_matrix = nbrs.kneighbors_graph(points, k, mode='connectivity').tocoo()
    edge_index = torch.tensor(np.vstack([adjacency_matrix.row, adjacency_matrix.col]), dtype=torch.long)

    data = Data(x=x, edge_index=edge_index, edge_attr=torch.tensor([1]*edge_index.shape[1], dtype=torch.float), y=torch.tensor([label]))

    return data, img, kp

def draw_keypoints_on_image(img, keypoints, output_path):
    img_kp = img.copy()
    for kp in keypoints:
        pt = tuple(map(int, kp.pt))
        cv2.circle(img_kp, pt, radius=2, color=(0, 0, 0), thickness=-1)
    cv2.imwrite(output_path, img_kp)
    print(f"Saved: {output_path}")

def draw_graph_on_image(img, keypoints, edge_index, output_path):
    img_graph = img.copy()
    for i in range(edge_index.shape[1]):
        pt1 = tuple(map(int, keypoints[edge_index[0, i]].pt))
        pt2 = tuple(map(int, keypoints[edge_index[1, i]].pt))
        cv2.line(img_graph, pt1, pt2, color=(0, 0, 0), thickness=1)
    for kp in keypoints:
        pt = tuple(map(int, kp.pt))
        cv2.circle(img_graph, pt, radius=2, color=(0, 0, 0), thickness=-1)
    cv2.imwrite(output_path, img_graph)
    print(f"Saved: {output_path}")

if __name__ == "__main__":
    input_dir = r"D:\CODES\Task1\HeathlandSpeciesClassifier\image_2_graph_samples"  # 🔁 Replace with your folder path
    output_dir = r"D:\CODES\Task1\HeathlandSpeciesClassifier\image_2_graph_samples\graph_overlay_output"
    os.makedirs(output_dir, exist_ok=True)

    # Match jpg, png, jpeg files
    image_paths = glob(os.path.join(input_dir, "*.[jpJP][pnPN]*[gG]"))

    for image_path in image_paths:
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        label = 0  # 🔁 Optional: Replace with real label if needed

        try:
            data, img, keypoints = keypoint_graph(image_path, label)

            if keypoints:
                keypoints_path = os.path.join(output_dir, f"{base_name}_keypoints.jpg")
                draw_keypoints_on_image(img, keypoints, keypoints_path)

                if data.edge_index.shape[1] > 0:
                    graph_path = os.path.join(output_dir, f"{base_name}_graph.jpg")
                    draw_graph_on_image(img, keypoints, data.edge_index, graph_path)
                else:
                    print(f"No edges in graph: {base_name}")
            else:
                print(f"No keypoints found in: {base_name}")
        except Exception as e:
            print(f"Error processing {image_path}: {e}")


Saved: D:\CODES\Task1\HeathlandSpeciesClassifier\image_2_graph_samples\graph_overlay_output\im117_117_8.1.0_keypoints.jpg
Saved: D:\CODES\Task1\HeathlandSpeciesClassifier\image_2_graph_samples\graph_overlay_output\im117_117_8.1.0_graph.jpg
Saved: D:\CODES\Task1\HeathlandSpeciesClassifier\image_2_graph_samples\graph_overlay_output\im117_118_4.1.0_keypoints.jpg
Saved: D:\CODES\Task1\HeathlandSpeciesClassifier\image_2_graph_samples\graph_overlay_output\im117_118_4.1.0_graph.jpg
Saved: D:\CODES\Task1\HeathlandSpeciesClassifier\image_2_graph_samples\graph_overlay_output\im1_70_6.1.0_keypoints.jpg
Saved: D:\CODES\Task1\HeathlandSpeciesClassifier\image_2_graph_samples\graph_overlay_output\im1_70_6.1.0_graph.jpg
Saved: D:\CODES\Task1\HeathlandSpeciesClassifier\image_2_graph_samples\graph_overlay_output\im1_77_9.1.0_keypoints.jpg
Saved: D:\CODES\Task1\HeathlandSpeciesClassifier\image_2_graph_samples\graph_overlay_output\im1_77_9.1.0_graph.jpg
Saved: D:\CODES\Task1\HeathlandSpeciesClassifier\ima